# Manipulating Molecular Simulation Data with MDAnalysis

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>
**Authors**:

- *Creation*: Dr Micaela Matta (micaela.matta@kcl.ac.uk), Dr Richard Gowers (richardjgowers@gmail.com), Dr Irfan Alibay (ialibay@gmail.com)
- *Updates*: Dr Antonia Mey (antonia.mey@ed.ac.uk), Dr Matteo Degiacomi (matteo.t.degiacomi@durham.ac.uk)

This notebook is adapted from materials developed for the [2022 CCPBioSim Workshop](https://github.com/MDAnalysis/WorkshopMDMLEdinburgh2022), which was in turn adapted from the [2021 PRACE Workshop](https://github.com/MDAnalysis/WorkshopPrace2021) and the [2018 Workshop/Hackathon](https://github.com/MDAnalysis/WorkshopHackathon2018).

**Learning Objectives**:
* How to load your molecular simulation data into MDAnalysis
* Basic features of the `MDAnalysis.Universe`
* Working with `AtomGroup`s
* Visualise your simulation data with `nglview`
* Select atoms of interest using `select_atoms`
* Iterating through your trajectory to extract quantities of interest

**Jupyter cheat sheet:**
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

<div class="alert alert-info"><b> Remember: variables persist between cells</b> 
    
Be aware that it is the order of execution of cells that is important in a Jupyter notebook, not the <em>order</em> in which they appear. Python will remember <em>all</em> the code that was run previously, including any variables you have defined, irrespective of the order in the notebook. Therefore if you define variables lower down the notebook and then (re)run cells further up, those defined further down will still be present. </div> 

## Table of Contents

[1.   Fundamental MDAnalysis objects](#fundamentals)  
[2.   Selecting Atoms](#selections)    
[3.   Analysing MD simulation trajectories](#trajectory)    
[4.   Extra material: Bonds, angles, and dihedrals](#bonds)   
[5.   Conclusion](#conclusion)    

## 0. Google Colab setup
<div class="alert alert-warning">
<b>Attention:</b> Please only run the following cells if you are using Colab! These cells install necessary packages and download data.</div>

In [ ]:
!if [ -n "$COLAB_RELEASE_TAG" ]; then pip install condacolab; fi
import condacolab
condacolab.install()

import condacolab
!mamba install -c conda-forge mdanalysis mdanalysistests mdanalysisdata nglview scikit-learn ipywidgets=7.6.0

In [ ]:
# enable third party jupyter widgets
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!if [ -n "$COLAB_RELEASE_TAG" ]; then git clone https://github.com/CCPBioSim/CCP5_Simulation_of_BioMolecules; fi
import os
os.chdir(f"CCP5_Simulation_of_BioMolecules{os.sep}5_Analysis_MDAnalysis")

## 1. Fundamental MDAnalysis objects
<a id='fundamentals'></a>

### Universe loading 101

> "*If you wish to make an apple pie from scratch, you must first invent the Universe.*" 
> ~ Carl Sagan

First, we need to import `MDAnalysis`, giving us access to all the components in its namespace:

In [ ]:
import MDAnalysis as mda

One of the most fundamental objects in the `MDAnalysis` data model is the `Universe` object.
A `Universe` can be thought of as an interface to all the data of a simulation;
it contains all of a simulations' topology information (names, charges, masses etc) at the least,
but usually also includes trajectory information (positions, velocities, etc.) as well.

In order to do anything, we do need some actual molecular dynamics data to work with. Let's load an example (the protein adenylate kinase, or AdK) from the MDAnalysis tests data:

In [ ]:
from MDAnalysis.tests.datafiles import PSF, DCD

To make a `Universe`, we need at the very least a topology file - see the [topology readers](https://docs.mdanalysis.org/2.0.0-dev0/documentation_pages/topology/init.html) documentation for a list of supported file formats. 

Since the type of topology file we are using in this example (a PSF file) does not contain coordinates, we will also need to load a trajectory file (in this case a DCD file) so we have some position data to work with later.

In [ ]:
u = mda.Universe(PSF, DCD)

print(u)

### AtomGroups

We now have a `Universe` object. Since the topology (PSF) file we loaded contained both atom identities and bond information, the `Universe` is able to access these details.



We can access all atoms in the `Universe` through the `Universe.atoms` attribute.
This returns an `AtomGroup`, which is probably the most important class we will learn about.

In [ ]:
ag = u.atoms
type(ag)

An `AtomGroup` is like an array of atoms, and offers access to the data of these atoms through various attributes:

In [ ]:
ag.indices

In [ ]:
ag.names

In [ ]:
ag.resnames

In [ ]:
ag.resids

In [ ]:
ag.charges

In [ ]:
ag.masses

In [ ]:
ag.types

All of these attributes of an `AtomGroup` return numpy arrays of the same length as the `AtomGroup` itself;
that is, each element corresponds to each atom in the `AtomGroup`, in order.

In [ ]:
print(ag.n_atoms)
print(len(ag.names))

In general, `MDAnalysis` will try and extract as much information as possible from the files given to `Universe`. The [topology readers](https://docs.mdanalysis.org/2.0.0-dev0/documentation_pages/topology/init.html) documentation lists the attributes that are extracted from each filetype.

### Working with individual atoms

By slicing an `AtomGroup` we can access individual `Atom` objects.
These `Atom` objects will have singular versions of the various attributes of `AtomGroup`s.

In general working with individual `Atom` objects is discouraged as it is inefficient and will lead to poor performance.

In [ ]:
a = u.atoms[0]
print(a)

In [ ]:
print("name:", a.name)
print("resid:", a.resid)
print("resname:", a.resname)

### ResidueGroups and SegmentGroups

The `Universe` also gives higher-order topology objects, including `ResidueGroups` and `SegmentGroups`. We can access all residues in the `Universe` with:

In [ ]:
u.residues

And all segments with:

In [ ]:
u.segments

`ResidueGroups` and `SegmentGroups` also behave similarly to `AtomGroups`, with many of their methods returning `numpy` arrays with each element corresponding to a single residue or segment, respectively.

In [ ]:
u.residues.resnames

In [ ]:
u.segments.segids

You can also create a `ResidueGroup` from an `AtomGroup`:

In [ ]:
ag.residues

<div class="alert alert-success">
<b>Task 1a. </b> Load the GRO topology file from <code>MDAnalysis.tests.datafiles</code> and count how many atoms, residues and segments it contains.
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

from MDAnalysis.tests.datafiles import GRO
u_gro = mda.Universe(GRO)
print('Atoms: ', u_gro.atoms.n_atoms)
print('Residues: ', u_gro.residues.n_residues)
print('Segments: ', u_gro.segments.n_segments)

```
</details>

<div class="alert alert-success">
<b>Task 1b. </b>  From the above Universe, find the name of the:
    <ul>
    <li>first segment</li>
    <li>last atom</li>
    <li>10th residue</li>
    </ul>
</div>


In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

print('First segment:', u_gro.segments[0].segid)
print('Last atom: ', u_gro.atoms[-1].name)
print('10th residue: ', u_gro.residues[9].resname)

```
</details>

<a id='selections'></a>
## 2. Selecting atoms

It is commonplace to operate on a subset of atoms in the system.
`MDAnalysis` offers a few different ways to select atoms, in this section we will go over the most useful ones.

### Numpy-style selections

As previously mentioned, an `AtomGroup` is like an array of atoms, and therefore we can slice it exactly like we would slice a `numpy` array.

The simplest option to select specific atom is to use "fancy indexing". You can specify the atoms in a list:

In [ ]:
u = mda.Universe(PSF, DCD)
u.atoms[[1, 4, 5 , 0]]

or as a range:

In [ ]:
u.atoms[1:10]

You can also create a boolean array (containing `True`/`False` values) of the same length as the `AtomGroup`. Every atom for which the array is set to `True` will be selected. Here is an example of how we can create such a boolean array:

In [ ]:
selection_ar = u.atoms.resnames == 'GLY'
print("selection array = ", selection_ar)
u.atoms[selection_ar]

We can also do this with `ResidueGroup`s and `SegmentGroup`s, e.g.:

In [ ]:
u.residues[u.residues.resnames == 'GLY']

### Selection Strings and `select_atoms`

`MDAnalysis` also features means to select atoms via a string of text, which is often more convenient. This is done via the `select_atoms` command.

To see the available keywords of `select_atoms`we can consult the [online documentation](https://docs.mdanalysis.org/2.0.0-dev0/documentation_pages/selections.html). This information can also be directly obtained in this notebooks as follows:

In [ ]:
u.atoms.select_atoms?

For example, we can select all glycines by:

In [ ]:
glycines = u.select_atoms("resname GLY")
glycines

If selecting by residue number, we can specify a range:

In [ ]:
first10resids = u.select_atoms("resid 1-10")
first10resids

We can use `and`/`or`/`not` with [logical conjunctions](https://en.wikipedia.org/wiki/Logical_conjunction):

In [ ]:
acidic = u.select_atoms("resname GLU or resname ASP")
acidic

For name like selections, we can also Unix shell-style wildcards such as `*`. Here for example `name OD*` would select atoms named `OD1, OD2, OD3` etc:

In [ ]:
acidic_o = acidic.select_atoms('name OD* or name OE*')
acidic_o

As a shortcut, multiple values can be given and these will be implicitly OR'd together.
To select all atoms with name NZ or NH* in residues named LYS or ARG:

In [ ]:
basic_n = u.select_atoms("(resname LYS ARG) and (name NZ NH*)")
basic_n

There are also several preset keywords for useful selections such as `backbone`, which selects all CA, C, O and N atoms:

In [ ]:
backbone_1 = u.select_atoms('backbone')
backbone_2 = u.select_atoms('name CA C O N')
backbone_1 == backbone_2

### Geometric selections

The `select_atoms` method also has various geometric keywords that make selecting atoms based on geometric criteria much easier.

For example, we can look for salt bridges by using the `around` selection operator to specify only atoms within 4 $\unicode{x212B}$ of a particular selection. Note that we can simplify selection strings by referring to previous `AtomGroups`.

In [ ]:
acidic = u.select_atoms("group acidic and around 4 group basic", acidic=acidic_o, basic=basic_n)
acidic

You can also select atoms based on absolute position using `prop`, e.g.:

In [ ]:
upper_z = u.select_atoms('prop z > 10')
upper_z

AtomGroups can be concatenated with `+`, and subtracted with `-`. For instance, the following two selections are also identical: 

In [ ]:
no_H1 = u.atoms.select_atoms('resname LYS ARG and not name H*')
no_H2 = u.atoms.select_atoms('resname LYS ARG') - u.atoms.select_atoms('name H*')

no_H1 == no_H2

<div class="alert alert-warning">
By design, an `AtomGroup` can have repeats of the same atom, for example through this selection: <code> ag = u.atoms[[0, 0, 1, 2, 4, 4, 5]]</code>.
The `unique` property will return a version of the `AtomGroup` with only one of each Atom: <code> ag.unique </code>
</div>

<div class="alert alert-success">
<b>Task 2a. </b> Select residues 100 to 200, first using indexing and then using a selection string, and confirm you get the same selection.
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

ag1 = u.residues[99:200]
ag2 = u.select_atoms("resid 100-200").residues
ag1 == ag2

```
</details>

<div class="alert alert-success">
<b>Task 2b. </b> Count the number of Arginine (ARG) residues.
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

ag = u.select_atoms('resname ARG')
print(len(ag.residues))
```
</details>

<div class="alert alert-success">
<b>Task 2c: </b> Select all Nitrogen atoms within 5.0 $\unicode{x212B}$ of an alpha carbon atom.
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

ag = u.select_atoms('name N* and around 5 name CA')
print(ag)
```
</details>

And now we can launch the viewer:

## 3. Analysing MD simulation Trajectories
<a id='trajectory'></a>

### Loading a trajectory

Loading a trajectory is done in the same way as loading any type of coordinates (as shown in session 1). All you have to do is create a `Universe` object by passing it a topology and the trajectory (here in this case a PSF file and DCD trajectory respectively).

In [ ]:
# First let's load a PSF and DCD from the MDAnalysis test data
from MDAnalysis.tests.datafiles import PSF, DCD
import MDAnalysis as mda
u = mda.Universe(PSF, DCD)

Something useful we can do before getting started, is visualizing the trajectory. This can be done using the [nglview](https://github.com/nglviewer/nglview#usage) package, that allows to visualise a MDAnalysis `Universe` or `AtomGroup` directly on the Jupyter notebook. First we load the structure:

In [ ]:
import nglview as nv

view_u = nv.show_mdanalysis(u)
view_u.camera = 'orthographic'

Then we visualize it!

In [ ]:
view_u

Trajectory functionality is centered around the `Universe.trajectory` object.

In [ ]:
u.trajectory

This `trajectory` object has a length in `frames` and a time unit of **picoseconds** (more information about the [MDAnalysis base units](https://docs.mdanalysis.org/2.0.0-dev0/documentation_pages/units.html#id4) is in the docs). It has many useful attributes, such as the the number of frames `n_frames`, the time between frames `dt`, the total trajectory time `totaltime`.

In [ ]:
print(u.trajectory.n_frames) # print the number of frames
print(u.trajectory.dt) # We can get the time between frames with `dt`
print(u.trajectory.totaltime) # And the total simulation time from `totaltime`

### The timestep object

One of the key components of trajectories is the *Timestep* object `ts`. This is the object that holds the trajectory information **specific to the current frame**.

This information mainly includes:
* The frame number and time
* Unitcell dimensions as `[A, B, C, alpha, beta, gamma]` (or `None` if not available)
* The positions (also forces and/or velocities if available)

In [ ]:
u.trajectory.ts

### Moving through a trajectory

Up until this point, we have primarily been inspecting only a single frame of the `trajectory` object. By default when creating a `Universe`, the *Timestep* is loaded with the information from the first (zero-th) frame in the trajectory.

Here we look at how we can traverse through the trajectory and access the data from different frames.

We can consider the `trajectory` object to be an iterator that loads trajectory data from a source (i.e. in most cases the input trajectory file), and feeds the relevant data to the *Timestep* object.

The following operations can be done to access the trajectory:
* Random access via trajectory indexing
* Iterating over all frames
* Slicing to iterate over a sub-section of the trajectory


<div class="alert alert-info"><b>Reminder</b>: As is standard in Python, `trajectory` access is done via <b>0-based indices</b>. So the first frame is <code>0</code>, and the final frame is <code>n_frames - 1</code>. 
</div>

### Trajectory indexing

It is possible to randomly access any frame along a trajectory by passing the index of the frame to the trajectory.

In [ ]:
# Let's create an atomgroup for the first two atoms in the Universe
# and check their current position at frame 0
first_two_atoms = u.atoms[:2]
print('current frame: ', u.trajectory.frame)
print(first_two_atoms.positions)

In [ ]:
# Now let's move to the 7th frame and check that the atoms position was updated
u.trajectory[6]
print('current frame: ', u.trajectory.frame)
print(first_two_atoms.positions)

<div class="alert alert-info">
<b>Reminder</b>: <code>AtomGroup</code>s are not static objects! While the atoms they represent do not change, their positions (and forces or velocities if available) will change as you move through the trajectory.
</div>

### Extracting properties by iterating through the trajectory

Iterating through a trajectory is the most common way to move through a trajectory. For example one could access every frame in the trajectory and store the current time using the following:

In [ ]:
# Create a list for the times
times = []
for ts in u.trajectory:
    times.append(u.trajectory.time)
    
print(times)      

Selecting a start and end frame, and a stride through them, can be done with normal <code>numpy</code> syntax.  Let's slice starting at the second frame, ending on the before last frame.

In [ ]:
times_stride = []
for ts in u.trajectory[1:-2:2]:
    times_stride.append(u.trajectory.time)

print(times_stride)

We will now loop through the simulation to calculate the protein's radius of gyration

In [ ]:
bb = u.select_atoms('protein')
rg = []
for ts in u.trajectory:  
    rgyr = bb.radius_of_gyration()   
    print(f"frame = {u.trajectory.time}: Rgyr = {rgyr} A")
    rg.append(rgyr)

Let's now plot what we have calculated.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.plot(times, rg)
ax.set_xlabel("time (ps)")
ax.set_ylabel("Rgyr ($\AA$)");

<div class="alert alert-success">
<b>Task 3. </b> Can you modify the loop above, calculating the radius of gyration, to measure the end-to-end distance of the protein? <br>
Hint: The N-terminal Nitrogen is the first atom in the array <code>u.select_atoms('name N')[0]</code>.
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

nterm = u.select_atoms('name N')[0]
cterm = u.select_atoms('name C')[-1]
dist = []
for ts in u.trajectory:
    r = cterm.position - nterm.position # end-to-end vector from atom positions
    d = numpy.linalg.norm(r)  # end-to-end distance
    print("frame = {0}: d = {1} A".format(ts.frame, d))
    dist.append(d)

dist = np.array(dist)
fig, ax = plt.subplots(1, 1)
ax.plot(dist[:, 0], dist[:, 1])
ax.set_xlabel("time (ns)")
ax.set_ylabel("end-to-end-distance ($\AA$)")
```
</details>

Some quantities can be extracted from the trajectory without the need of looping over them. There are methods available that do the looping for you! One such method enables calculating the RMSD of every simulation frame with respect to a frame of interest. The <code>MDAnalysis.analysis</code> module contains plenty of useful data analysis methods ([see here]([User Guide](https://userguide.mdanalysis.org/2.0.0-dev0/)). Let's load the <code>rms</code> sub-module, and let's have a look at the parameters of the <code>RMSD</code> method.

In [ ]:
from MDAnalysis.analysis import rms
rms.RMSD?

the <code>RMSD</code> method aligns all frames in a simulation vs a reference (using atoms defined by the <code>select</code> parameter). It then calculates the RMSD according to one or more atom selections (by default including those listed in the <code>select</code> parameters, plus any described in the <code>groupselections</code> optional parameter).

In [ ]:
R_all = rms.RMSD(u, u, select="backbone", groupselections=["backbone and resid 1-136", "backbone and not resid 1-136"])
R_all.run()

Let's now plot all the data we have gathered. Note that simulation timesteps are also included in the dataset produced.

In [ ]:
rmsd = R_all.rmsd.T  # transpose makes it more comfortable for plotting
time = rmsd[1]
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(time, rmsd[2], '-', c="gray", label="all")
ax.plot(time, rmsd[3], '-', c="mediumblue", label="Selection 1")
ax.plot(time, rmsd[4], '-', c="deepskyblue", label="Selection 2")
ax.legend(loc="best", frameon=False)
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"RMSD ($\AA$)")

## 4. Extra material: bonds, angles, and dihedrals
<a id='bonds'></a>

`MDAnalysis` enables getting connectivity information between atoms, such as bonds, angles, and dihedrals.
These are read from the input topology file,
with angle and dihedrals inferred from the bonds present,
i.e. there is a measurable angle between two bonds with a common atom.

For example in a PDB file CONECT records are read as bonds,
however if none are present...

In [ ]:
from MDAnalysis.tests.datafiles import PDB_small

In [ ]:
# NBVAL_RAISES_EXCEPTION
u = mda.Universe(PDB_small)

u.bonds

### Guessing bonds

Bonds are typically read form the topology file,
however if they are not present it is possible to guess these by passing the `guess_bonds=True` keyword argument to `Universe` creation.
This algorithm guesses bonds that are present based upon the positions of particles and their assumed radius.

In [ ]:
u = mda.Universe(PDB_small, guess_bonds=True)

In [ ]:
u.atoms.bonds

In [ ]:
u.atoms.angles

In [ ]:
u.atoms.dihedrals

Like AtomGroups, these can be sliced to yield individual items:

In [ ]:
u.atoms.bonds[3]

All of these give the associated measurement, bond length or angle, via the `values()` or `value()` method:

In [ ]:
u.atoms.bonds.values()

In [ ]:
u.bonds[2].value()

### Selecting main chain dihedrals

For polypeptides, it is also possible to select the atoms involved in the $\varphi$ (phi), $\psi$ (psi) and $\omega$ (omega) dihedrals.
These are acessed via the `phi_selection`, `psi_selection` and `omega_selection` methods of a **Residue**.

In [ ]:
r = u.residues[1]
print(r)

In [ ]:
print(r.phi_selection())
print(r.psi_selection())
print(r.omega_selection())

These methods return `AtomGroup`s.  To turn a 4-member group into a dihedral, the `.dihedral` converting property can be used:

In [ ]:
r.phi_selection().dihedral

There are similar properties for casting **any** 2 atoms into a `Bond` (`AtomGroup.bond`) and casting **any** 3 atoms into an `Angle` (`AtomGroup.angle`).

In [ ]:
u.atoms[[0, 2]].bond

<div class="alert alert-success">
<b>Task 4. </b> Calculate the angle between the C$\alpha$ atoms in the fifth, sixth, and seventh residue.
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

at1 = u.residues[4].atoms.select_atoms('name CA')
at2 = u.residues[5].atoms.select_atoms('name CA')
at3 = u.residues[6].atoms.select_atoms('name CA')

ag = at1 + at2 + at3

ag.angle.value()
```
</details>

## 5. Conclusion
<a id='bonds'></a>

<div class="alert alert-info">
<b>Key points:</b>

- Start any analysis by loading your protein structure or simulation in a `Universe`
- `AtomGroup`s are selection of atoms that can be obtained using numpy syntax, or selection strings
- `AtomGroup` content (positions, forces, velocities) change as you move through the trajectory
- `nglview` enables visualizing a Universe.
</div>

This has been a brief overview of `MDAnalysis`. This is a vast package, featuring most of the typical analysis you may want to carry out, and plenty of freedom to produce your own.

**Next steps**
- More information on how to extract information from trajectory, building upon and extending what we have seen here, is available in the Jupyter notebook [Extra_Trajectories.ipynb](Extra_Trajectories.ipynb)
- If you are interested in knowing how the images in the Lecture slides were produced, have a look at the notebook [Extra_p24_analysis.ipynb](Extra_p24_analysis.ipynb)
- [nglview](https://github.com/nglviewer/nglview#usage) has a lot of visualisation options - you can add multiple selections to one view, change their colour and representation style and more. Look through their documentation and see what you can create!

**Additional resources**
- For more on how to use MDAnalysis, see the [User Guide](https://userguide.mdanalysis.org/2.0.0-dev0/) and [documentation](https://docs.mdanalysis.org/2.0.0-dev0/)
 - Ask questions on the [user mailing list](https://groups.google.com/group/mdnalysis-discussion) or on [Discord](https://discord.gg/fXTSfDJyxE)
 - Report bugs on [GitHub](https://github.com/MDAnalysis/mdanalysis/issues?)


---